In [ ]:
from pathlib import Path
import argparse, os, json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

ROOT_PATH = os.getcwd()
DATASET_PATH = os.path.join(os.getcwd(), "sample_dataset")
YOLO_WORKSPACE_PATH = os.path.join(os.getcwd(), "yolov7")

## Training sample dataset

In [7]:
yaml_bee_file = """
# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../sample_dataset/train/
val: ../sample_dataset/valid/

# number of classes
nc: 1

# class names
names: [ "Honeybee" ]
"""

bee_data = os.path.join(YOLO_WORKSPACE_PATH, "bee_data_coco.yaml")
f = open(bee_data, "w")
f.write(yaml_bee_file)
f.close()

In [ ]:
## create network arch config file -> yolo7 tiny
yaml = """
# parameters
nc: 1  # number of classes
depth_multiple: 1.0  # model depth multiple
width_multiple: 1.0  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# yolov7-tiny backbone
backbone:
  # [from, number, module, args] c2, k=1, s=1, p=None, g=1, act=True
  [[-1, 1, Conv, [32, 3, 2, None, 1, nn.LeakyReLU(0.1)]],  # 0-P1/2  
  
   [-1, 1, Conv, [64, 3, 2, None, 1, nn.LeakyReLU(0.1)]],  # 1-P2/4    
   
   [-1, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 7
   
   [-1, 1, MP, []],  # 8-P3/8
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 14
   
   [-1, 1, MP, []],  # 15-P4/16
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 21
   
   [-1, 1, MP, []],  # 22-P5/32
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [512, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 28
  ]

# yolov7-tiny head
head:
  [[-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, SP, [5]],
   [-2, 1, SP, [9]],
   [-3, 1, SP, [13]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -7], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 37
  
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [21, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]], # route backbone P4
   [[-1, -2], 1, Concat, [1]],
   
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 47
  
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [14, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]], # route backbone P3
   [[-1, -2], 1, Concat, [1]],
   
   [-1, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [32, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [32, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 57
   
   [-1, 1, Conv, [128, 3, 2, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, 47], 1, Concat, [1]],
   
   [-1, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [64, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [64, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 65
   
   [-1, 1, Conv, [256, 3, 2, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, 37], 1, Concat, [1]],
   
   [-1, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-2, 1, Conv, [128, 1, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [-1, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [[-1, -2, -3, -4], 1, Concat, [1]],
   [-1, 1, Conv, [256, 1, 1, None, 1, nn.LeakyReLU(0.1)]],  # 73
      
   [57, 1, Conv, [128, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [65, 1, Conv, [256, 3, 1, None, 1, nn.LeakyReLU(0.1)]],
   [73, 1, Conv, [512, 3, 1, None, 1, nn.LeakyReLU(0.1)]],

   [[74,75,76], 1, Detect, [nc, anchors]],   # Detect(P3, P4, P5)
  ]
"""
bee_arch = os.path.join(YOLO_WORKSPACE_PATH, "bee_arch.yaml")
f = open(bee_arch, "w")
f.write(yaml)
f.close()

In [58]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
%cd yolov7
!python train.py --workers 8 --device 0 --epoch 50 --batch-size 2 --data bee_data_coco.yaml --img 640 --cfg bee_arch.yaml --weights 'yolov7_training.pt' --name test_result --hyp data/hyp.scratch.tiny.yaml --project test

In [54]:
!python test.py --data bee_data_coco.yaml --img 640 --batch 2 --conf 0.001 --iou 0.65 --device 0 --weights test/test_result9/weights/best.pt

## Get Prediction.json file

In [11]:
import fnmatch
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

pred_json = find("best_predictions.json", os.getcwd())[-1]
print(pred_json)

c:\Users\yonghoon\Desktop\cocoeval_with_yolov7\yolov7\runs\test\exp2\best_predictions.json


In [13]:
pred = json.load(open(pred_json, 'r'))
print(pred[0])

{'image_id': 'Screenshot-from-2021-12-29-17-34-45_png.rf.84d5c5d84d99aa31b105afd25cb1748f', 'category_id': 1, 'bbox': [0.0, 384.15, 30.002, 19.5], 'score': 0.12683}


## Get Annotation.json file

In [46]:
import shutil
valid = os.path.join(DATASET_PATH, "valid")
merge = os.path.join(valid, "merge")
## merge
if ( not os.path.exists(merge) ):
    os.makedirs(merge)
%cd $valid
txt = "Honeybee"
f = open('obj.names', 'w')
f.write(txt)
f.close()
!ls
!cp -r images/*.jpg merge/
!cp -r labels/*.txt merge/

c:\Users\yonghoon\Desktop\cocoeval_with_yolov7\sample_dataset\valid
images
labels
labels.cache
merge
obj.names
valid.txt


In [47]:
CONVERTER_PATH = os.path.join(ROOT_PATH, "Yolo-to-COCO-format-converter")
%cd $CONVERTER_PATH

c:\Users\yonghoon\Desktop\cocoeval_with_yolov7\Yolo-to-COCO-format-converter


In [48]:
valid_txt = os.path.join(valid, "valid.txt")
!python path_replacer.py --path_image_folder $merge --path_txt $valid_txt

----File ".txt" created successfully----


In [53]:
## important, main.py classes modify <-- !! -->
!python main.py --path $valid_txt --output anno.json
!cat output/anno.json

In [51]:
## image id mapping
valid_json_file = json.load(open("output/anno.json", 'r'))
image_id_filename_mapping = {}
val_images = valid_json_file["images"]
for i in range(len(val_images)):
    image_id_filename_mapping[val_images[i]["file_name"]] = val_images[i]["id"]  

def getFilenameID(fname):
    return image_id_filename_mapping[f"{fname}.jpg"]

for i in range(len(pred)):
    image_id = getFilenameID(pred[i]["image_id"])
    pred[i]["image_id"] = image_id
    pred[i]["category_id"] = 1

with open("test.json", "w") as outfile:
    outfile.write(json.dumps(pred, indent=4))

test = os.path.join("test.json")

In [57]:
%cd $ROOT_PATH
!python get_coco_metric.py --annotation $CONVERTER_PATH/"output/anno.json" --prediction $CONVERTER_PATH/"test.json"

c:\Users\yonghoon\Desktop\cocoeval_with_yolov7
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.68s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.045
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.017
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.074
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.004
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.026
 Average Recall   